**Construcción de dataset de análisis**

In [1]:
import pandas as pd

dataset1 = "viajeros-hospedados-residentes-y-no-residentes-por-destino.csv"
datos1 = pd.read_csv(dataset1)
cant_viajeros_res_nores = datos1.iloc[:, :4]

In [2]:
dataset2 = "estadia-media-residentes-y-no-residentes-por-destino.csv"
datos2 = pd.read_csv(dataset2)
estadia_media_res_nores = datos2.iloc[:, :4]

In [5]:
dataset3 = "tasas-de-ocupacion-plazas-por-region-y-categoria.csv"
datos3 = pd.read_csv(dataset3)
tasas_ocup_plazas = datos3.iloc[:, :4]

In [7]:
dataset4 = "tasas-de-ocupacion-habitacion-por-region-y-categoria.csv"
datos4 = pd.read_csv(dataset4)
tasas_ocup_habit = datos4.iloc[:, :4]

In [9]:
dataset5 = "facturacion-total-en-pesos-por-region-y-categoria.csv"
datos5 = pd.read_csv(dataset5)
factu_region_categ = datos5.iloc[:, :4]

In [11]:
data1 = cant_viajeros_res_nores.merge(estadia_media_res_nores, on=['indice_tiempo', 'region_de_destino', 'origen_viajeros'], how='outer')
data1.head()

,indice_tiempo,region_de_destino,origen_viajeros,viajeros,estadia_media_en_noches
0,2007-01-01,Buenos Aires,Residentes,401221.0,5.3
1,2007-01-01,Buenos Aires,No residentes,12889.0,3.8
2,2007-01-01,CABA,Residentes,139181.0,2.3
3,2007-01-01,CABA,No residentes,196750.0,2.4
4,2007-01-01,Cuyo,Residentes,125410.0,2.7


In [13]:
data2 = tasas_ocup_plazas.merge(tasas_ocup_habit, on=['indice_tiempo', 'region_de_destino', 'categoria_del_hotel'], how='outer').merge(factu_region_categ, on=['indice_tiempo', 'region_de_destino', 'categoria_del_hotel'], how='outer')
data2.head()

,indice_tiempo,region_de_destino,categoria_del_hotel,tasa_de_ocupacion_plazas,tasa_de_ocupacion_habitaciones,facturacion_total_en_pesos
0,2012-01-01,Buenos Aires,1 y 2 estrellas,0.63,0.65,44811124.0
1,2012-01-01,Buenos Aires,3 estrellas/boutiques/aparts,0.67,0.72,88000600.0
2,2012-01-01,Buenos Aires,4 y 5 estrellas,0.62,0.67,68474622.0
3,2012-01-01,Buenos Aires,Para Hoteles,0.69,0.76,81220502.0
4,2012-01-01,Buenos Aires,Total,0.66,0.71,NaN


---------------------------------------

**Dataset definitivo**

In [15]:
data = data1.merge(data2, on=['indice_tiempo', 'region_de_destino'], how='outer')
data.tail()

,indice_tiempo,region_de_destino,origen_viajeros,viajeros,estadia_media_en_noches,categoria_del_hotel,tasa_de_ocupacion_plazas,tasa_de_ocupacion_habitaciones,facturacion_total_en_pesos
16795,2024-04-01,Patagonia,Total,NaN,2.3,1 y 2 estrellas,0.36,0.48,2.825963e+09
16796,2024-04-01,Patagonia,Total,NaN,2.3,3 estrellas/boutiques/aparts,0.29,0.39,6.432717e+09
16797,2024-04-01,Patagonia,Total,NaN,2.3,4 y 5 estrellas,0.36,0.49,1.579218e+10
16798,2024-04-01,Patagonia,Total,NaN,2.3,Para Hoteles,0.21,0.26,2.517031e+09
16799,2024-04-01,Patagonia,Total,NaN,2.3,Total,0.30,0.42,NaN


______________________________

**Dentro del periodo de las vacaciones de verano del 2020-2021, ¿qué tipo de turistas (residentes o
no residentes) tuvieron estadías más largas? ¿El levantamiento de las etapas de la cuarentena
ayudó a su incremento en el tiemp**o?

In [17]:
vacac_verano_20_21 = data[(data['indice_tiempo'] >= '2020-12-18') & (data['indice_tiempo'] <= '2021-02-01')]
vacac_verano_20_21.head()

,indice_tiempo,region_de_destino,origen_viajeros,viajeros,estadia_media_en_noches,categoria_del_hotel,tasa_de_ocupacion_plazas,tasa_de_ocupacion_habitaciones,facturacion_total_en_pesos
12600,2021-01-01,Buenos Aires,Residentes,169606.0,4.0,1 y 2 estrellas,0.28,0.31,200734296.0
12601,2021-01-01,Buenos Aires,Residentes,169606.0,4.0,3 estrellas/boutiques/aparts,0.33,0.38,574387225.0
12602,2021-01-01,Buenos Aires,Residentes,169606.0,4.0,4 y 5 estrellas,0.32,0.37,448107951.0
12603,2021-01-01,Buenos Aires,Residentes,169606.0,4.0,Para Hoteles,0.37,0.41,563676218.0
12604,2021-01-01,Buenos Aires,Residentes,169606.0,4.0,Total,0.33,0.37,NaN


Turismo verano 2020-2021

In [19]:
turis_estadias = vacac_verano_20_21.groupby('origen_viajeros')['estadia_media_en_noches']
prom_turistas_verano2020 = turis_estadias.mean()
prom_turistas_verano2020

origen_viajeros
No residentes    2.542857
Residentes       2.785714
Total            2.771429
Name: estadia_media_en_noches, dtype: float64

Turismo pre verano 2020

In [21]:
cuarent_2020 = data[(data['indice_tiempo'] >= '2020-03-20') & (data['indice_tiempo'] <= '2020-10-31')]
turismo_cuarent_20 = cuarent_2020.groupby('origen_viajeros')['estadia_media_en_noches']
prom_turistas_cuarent_2020 = turismo_cuarent_20.sum()
prom_turistas_cuarent_2020

origen_viajeros
No residentes    0.0
Residentes       0.0
Total            0.0
Name: estadia_media_en_noches, dtype: float64

**Tener en cuenta que el turismo comienza a tener normalidad entre noviembre y diciembre de 2020.

**Teniendo en cuenta el punto anterior, ¿cuál es la región del país que menos facturación tuvo durante
ese periodo? ¿Eran las mismas pre 2020? ¿Hubo algún cambio de preferencia**?

Regiones con menos recaudación en el verano 2020-2021

In [23]:
region_facturac = vacac_verano_20_21.groupby('region_de_destino')['facturacion_total_en_pesos']
region_fact_20_21_a = region_facturac.sum()
region_fact_20_21_b = region_fact_20_21_a.fillna(0)
region_fact = region_fact_20_21_b.sort_values(ascending=True)
region_fact.head()

region_de_destino
CABA       0.000000e+00
Norte      3.019155e+09
Cuyo       3.452416e+09
Litoral    5.069056e+09
Córdoba    7.392340e+09
Name: facturacion_total_en_pesos, dtype: float64

Regiones según recaudación pre 2020 (datos desde 2007)

In [25]:
fact_pre_2020 = data[(data['indice_tiempo'] <= '2020-03-20')] # REVER PERIODOS DE TIEMPO ELEGIDOS
aa = fact_pre_2020.groupby('region_de_destino')['facturacion_total_en_pesos']
bb = aa.sum()
cc = bb.fillna(0)
fact_region_pre_2020 = cc.sort_values(ascending=True)
fact_region_pre_2020.head()

region_de_destino
Córdoba         6.105507e+10
Norte           7.210058e+10
Cuyo            8.665739e+10
Buenos Aires    8.934292e+10
Litoral         1.218394e+11
Name: facturacion_total_en_pesos, dtype: float64

**¿Qué categorías de alojamiento fueron las más exitosas en ocupación entre 2022 y 2024?
¿Demuestra algún tipo de progreso económico o no**?

Ocupación 2022-2024

In [27]:
ocup_22_24 = data[(data['indice_tiempo'] >= '2022-01-01') & (data['indice_tiempo'] <= '2024-04-01')]

In [29]:
ocup_22_24.loc[:, 'viajeros'] = ocup_22_24['viajeros'].fillna(0)
ocup_22_24.loc[:, 'facturacion_total_en_pesos'] = ocup_22_24['facturacion_total_en_pesos'].fillna(0)

In [31]:
ocup_filtrado = ocup_22_24.groupby('categoria_del_hotel').agg({
    'facturacion_total_en_pesos': 'sum',
    'viajeros': 'sum'
}).reset_index()
ocup_filtrado['ocupacion'] = ocup_filtrado['facturacion_total_en_pesos'] / ocup_filtrado['viajeros']
ocup_2022_2024 = ocup_filtrado.sort_values(by='ocupacion', ascending=False)
ocup_2022_2024

,categoria_del_hotel,facturacion_total_en_pesos,viajeros,ocupacion
2,4 y 5 estrellas,3.200855e+12,46479560.0,68865.873149
1,3 estrellas/boutiques/aparts,1.211391e+12,46479560.0,26062.879323
3,Para Hoteles,5.796830e+11,46479560.0,12471.783430
0,1 y 2 estrellas,3.898394e+11,46479560.0,8387.329031
4,Total,0.000000e+00,46479560.0,0.000000


Ocupación 2020-2022

In [33]:
ocup_20_22 = data[(data['indice_tiempo'] >= '2020-01-01') & (data['indice_tiempo'] <= '2022-12-31')]

In [35]:
ocup_20_22.loc[:, 'viajeros'] = ocup_20_22['viajeros'].fillna(0)
ocup_20_22.loc[:, 'facturacion_total_en_pesos'] = ocup_20_22['facturacion_total_en_pesos'].fillna(0)

In [37]:
ocup_b_filtrado = ocup_20_22.groupby('categoria_del_hotel').agg({
    'facturacion_total_en_pesos': 'sum',
    'viajeros': 'sum'
}).reset_index()
ocup_b_filtrado['ocupacion'] = ocup_b_filtrado['facturacion_total_en_pesos'] / ocup_b_filtrado['viajeros']
ocup_2020_2022 = ocup_b_filtrado.sort_values(by='ocupacion', ascending=False)
ocup_2020_2022

,categoria_del_hotel,facturacion_total_en_pesos,viajeros,ocupacion
2,4 y 5 estrellas,7.428010e+11,35002445.0,21221.404142
1,3 estrellas/boutiques/aparts,2.982648e+11,35002445.0,8521.256206
3,Para Hoteles,1.511486e+11,35002445.0,4318.229207
0,1 y 2 estrellas,9.714372e+10,35002445.0,2775.341033
4,Total,0.000000e+00,35002445.0,0.000000


**Se observa crecimiento en la ocupación.

**Dentro de los registros pertenecientes al 2024, ¿cuál es la región más visitada por los turistas
residentes? En cuanto a ocupación de habitaciones, la elección fue pareja en las primeras tres regiones?**?

In [39]:
ocup_2024 = data[(data['indice_tiempo'] >= '2024-01-01') & (data['indice_tiempo'] <= '2024-04-01')]
ocup_x_region = ocup_2024[ocup_2024['origen_viajeros'] == 'Residentes'].groupby('region_de_destino')['viajeros'].sum()
ocup_x_region.sort_values(ascending=False)

region_de_destino
Patagonia       5242275.0
Buenos Aires    4653300.0
Córdoba         3960615.0
Litoral         3575030.0
CABA            3013910.0
Norte           2624265.0
Cuyo            2114400.0
Name: viajeros, dtype: float64

In [54]:
aloj_mejores_reg_2024 = ocup_2024[(ocup_2024['region_de_destino'] == 'Patagonia') | (ocup_2024['region_de_destino'] == 'Buenos Aires') | (ocup_2024['region_de_destino'] == 'Córdoba')]
aloj_mejores_reg_2024 = aloj_mejores_reg_2024[(aloj_mejores_reg_2024['origen_viajeros'] == 'Residentes')]
aloj_mejores_reg_2024 = aloj_mejores_reg_2024.groupby('categoria_del_hotel')['tasa_de_ocupacion_habitaciones'].sum()
aloj_mejores_reg_2024.sort_values(ascending=False)

categoria_del_hotel
4 y 5 estrellas                 5.91
Total                           5.72
3 estrellas/boutiques/aparts    5.64
Para Hoteles                    5.60
1 y 2 estrellas                 5.44
Name: tasa_de_ocupacion_habitaciones, dtype: float64